# Treinamento do modelo de prognóstico (autoencoder + HI_limit)

Este notebook prepara o modelo usado na etapa de **prognóstico de falhas**:

1. **Autoencoder** treinado com dados normais (janela de referência por cluster)
2. **HI_limit** calculado a partir da média da janela + regras Palavra-guia/Desvio (`prog_dados.csv`), salvo em `dados/HI_limit.txt`

## Fluxo

- **1** Carregar dados (`dados/dados.csv`), filtrar por cluster e janela `w`
- **2** Selecionar sensores (S1–S4), dividir em treino/teste
- **3** Normalizar com MinMaxScaler [-1, 1]
- **4** Definir e treinar o autoencoder
- **5** Calcular HI_limit (MSE na reconstrução dos valores “limite”) e salvar em `.txt`
- **6** Salvar modelo (`dados/autoencoder_unit3.h5`) e scaler (`dados/scaler.pkl`)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
from scipy.special import inv_boxcox
from scipy.special import boxcox as sp_boxcox
from scipy.stats import boxcox, shapiro, norm
from scipy.special import boxcox as boxcox_scalar

## 1. Carregamento e preparação dos dados

Carregamos `dados/dados.csv`, filtramos por **cluster** e tomamos as primeiras **w** amostras (janela de referência). As colunas de sensores (S1, S2, S3, S4) são usadas como entrada do autoencoder.

In [2]:
df = pd.read_csv("dados/dados.csv")




In [3]:
cluster = 1
w =  10

df= df[df["Cluster"] == cluster]

# Pegar os primeiros 10 pontos (Amostra 1 a 10)
df =  df.head(w)



In [4]:
df

,Cluster,Amostra,S1,S2,S3,S4
0,1,1,9.733,19.915,30.406,40.073
1,1,2,10.336,20.223,29.849,40.193
2,1,3,9.995,19.978,30.089,39.587
3,1,4,10.478,19.899,30.380,40.101
4,1,5,10.610,19.927,30.006,40.176
5,1,6,9.518,20.398,30.582,40.428
6,1,7,10.436,20.131,30.525,40.427
7,1,8,10.358,20.202,30.051,39.751
8,1,9,10.638,19.573,30.069,40.184
9,1,10,10.558,20.778,30.498,40.063


In [5]:
df = df[["S1", "S2", "S3", "S4"]]
df


,S1,S2,S3,S4
0,9.733,19.915,30.406,40.073
1,10.336,20.223,29.849,40.193
2,9.995,19.978,30.089,39.587
3,10.478,19.899,30.380,40.101
4,10.610,19.927,30.006,40.176
5,9.518,20.398,30.582,40.428
6,10.436,20.131,30.525,40.427
7,10.358,20.202,30.051,39.751
8,10.638,19.573,30.069,40.184
9,10.558,20.778,30.498,40.063


## 2. Seleção dos dados para treino

Mantemos apenas as colunas dos sensores (S1–S4). As primeiras **w** amostras (ex.: w=10) formam a janela de referência e serão usadas para treinar o autoencoder e calcular o HI_limit.

In [6]:
X = df
X

,S1,S2,S3,S4
0,9.733,19.915,30.406,40.073
1,10.336,20.223,29.849,40.193
2,9.995,19.978,30.089,39.587
3,10.478,19.899,30.380,40.101
4,10.610,19.927,30.006,40.176
5,9.518,20.398,30.582,40.428
6,10.436,20.131,30.525,40.427
7,10.358,20.202,30.051,39.751
8,10.638,19.573,30.069,40.184
9,10.558,20.778,30.498,40.063


## 3. Divisão dos dados

Divisão 80% treino / 20% teste para treinar e avaliar o autoencoder.

In [7]:
# Split into 80% training and 20% testing
X_train, X_test = train_test_split(
    X, test_size=0.2, random_state=42
)


## 4. Arquitetura do autoencoder

Rede simétrica: **encoder** reduz as 4 entradas (S1–S4) até o espaço latente (2 neurônios), o **decoder** reconstrói as 4 saídas. Ativação `tanh` em todas as camadas; dados normalizados em [-1, 1].

In [8]:
num_columns = df.shape[1]
num_columns

4

## 5. Normalização

`MinMaxScaler` no intervalo [-1, 1] para compatibilidade com `tanh`. O mesmo scaler é usado em treino, teste e no cálculo do HI_limit.

In [9]:
def prev_fib_smaller(n: int) -> int:
    a, b = 1, 1  # Fibonacci sequence: 1, 1, 2, 3, 5, ...
    while b <= n:
        a, b = b, a + b
    return a  # 'a' will be the last Fibonacci number <= n

num_cols = df.shape[1]
closest_smaller_fib = prev_fib_smaller(num_cols)
print(num_cols, "→", closest_smaller_fib)


4 → 3


## 6. Treinamento do autoencoder

Treino por MSE entre entrada e reconstrução. O modelo aprende a reconstruir bem o comportamento normal; dados anômalos geram maior erro (HI alto).

In [10]:
# Dimensions
input_dim = X_train.shape[1]  
encoding_dim = closest_smaller_fib             



## 7. Health Index (HI)

O HI é o MSE entre a entrada e a reconstrução do autoencoder. **HI baixo** = comportamento normal; **HI alto** = anomalia/degradação. Na etapa de teste, o HI é calculado para cada amostra e comparado ao HI_limit.

In [11]:
# -----------------------------------------
# Scale data for tanh (range [-1, 1])
# -----------------------------------------
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 8. Construção e compilação do autoencoder

Definição das camadas (encoder → gargalo 2D → decoder), compilação com perda MSE e exibição do resumo do modelo.

In [12]:
# -----------------------------------------
# Define autoencoder
# -----------------------------------------
input_data = Input(shape=(input_dim,))

# ---------- Encoder ----------
x = Dense(13, activation='tanh')(input_data)   # 1st hidden layer → 23
x = Dense(8, activation='tanh')(x)             # 2nd → 8
x = Dense(5, activation='tanh')(x)             # 3rd → 5
x = Dense(3, activation='tanh')(x)             # 4th → 3


# Bottleneck (latent space)
encoded = Dense(2, activation='tanh',
                activity_regularizer=regularizers.l1(1e-5))(x)


# ---------- Decoder ----------
x = Dense(3, activation='tanh')(encoded)       # 5th → 16
x = Dense(5, activation='tanh')(x)             # 6th → 32
x = Dense(8, activation='tanh')(x)             # 7th → 64
x = Dense(13, activation='tanh')(x)            # 8th → 128

# Output layer (reconstruction)
decoded = Dense(input_dim, activation='tanh')(x)  # 9th layer → 21 neurons

# ---------- Models ----------
autoencoder = Model(input_data, decoded)
encoder = Model(input_data, encoded)

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()



2026-02-10 16:09:03.980937: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2026-02-10 16:09:03.980981: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2026-02-10 16:09:03.980985: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.88 GB
2026-02-10 16:09:03.980999: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-10 16:09:03.981009: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 13)             │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │            18 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2)              │             8 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 5)              │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 13)             │           117 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │            56 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 498 (1.95 KB)

 Trainable params: 498 (1.95 KB)

 Non-trainable params: 0 (0.00 B)

## 9. Treinamento e salvamento

Treino do autoencoder (fit), reconstrução no conjunto de teste e salvamento do modelo em `dados/autoencoder_unit3.h5` e do scaler em `dados/scaler.pkl`.

In [13]:
# -----------------------------------------
# Train
# -----------------------------------------
autoencoder.fit(
    X_train_scaled, X_train_scaled,
    epochs=50,
    batch_size=64,
    shuffle=True,
    validation_data=(X_test_scaled, X_test_scaled),
    verbose=1
)

# -----------------------------------------
# Encode and decode
# -----------------------------------------
encoded_data = encoder.predict(X_test_scaled)
decoded_data_scaled = autoencoder.predict(X_test_scaled)

# Inverse transform to original scale
decoded_data = scaler.inverse_transform(decoded_data_scaled)

Epoch 1/50


2026-02-10 16:09:04.762180: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.5406 - val_loss: 1.0266
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - loss: 0.5331 - val_loss: 1.0163
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.5260 - val_loss: 1.0066
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - loss: 0.5193 - val_loss: 0.9973
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.5128 - val_loss: 0.9885
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 0.5066 - val_loss: 0.9802
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 0.5006 - val_loss: 0.9722
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 0.4948 - val_loss: 0.9646
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - loss: 0.4892 - val_loss: 0.9572
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 0.4836 - val_loss: 0.9501
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.4782 - val_loss: 0.9432
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - loss: 0.4727 - val_loss: 0.9365
Epo

In [14]:
import joblib


# after training
autoencoder.save("dados/autoencoder_unit3.h5")
joblib.dump(scaler, "dados/scaler.pkl")   # whatever you used to scale inputs


['dados/scaler.pkl']

## 10. HI_limit

Cálculo do **HI_limit** (limite de alerta):

1. Média dos sensores na janela de referência (X_train)
2. Regras por sensor em `dados/prog_dados.csv` (Palavra-guia: Maior/Menor; Desvio em %)
3. Valores modificados: Maior → média × (1 + desvio); Menor → média × (1 − desvio)
4. Limite ao range do scaler, normalização, passagem no autoencoder
5. **HI_limit** = MSE(entrada escalada, reconstrução)
6. Valor salvo em `dados/HI_limit.txt` para uso no notebook de teste.

In [18]:
df_prog = pd.read_csv("dados/prog_dados.csv")

# ---------- HI_limit: valor médio da janela de referência + regras (Palavra-guia e Desvio) ----------
valid_tags = list(X_train.columns)
mean_values = np.mean(X_train, axis=0)

col_start = 1 + (cluster - 1) * 4
col_end = col_start + 4
palavra_guia_row = df_prog.iloc[1, col_start:col_end]
desvio_row = df_prog.iloc[2, col_start:col_end]

def apply_rule_to_mean(mean_val, palavra_guia, desvio):
    if str(palavra_guia).strip().lower() in ('maior', 'mais'):
        return mean_val * (1 + desvio)
    return mean_val * (1 - desvio)

modified_values = np.zeros_like(mean_values, dtype=float)
for i in range(len(valid_tags)):
    pg = str(palavra_guia_row.iloc[i]).strip().lower()
    palavra_guia = 'Maior' if pg == 'mais' else 'Menor'
    d = float(desvio_row.iloc[i])
    if d > 1.0:
        d = d / 100.0
    modified_values[i] = apply_rule_to_mean(mean_values[i], palavra_guia, d)

if hasattr(scaler, 'data_min_') and hasattr(scaler, 'data_max_'):
    for i in range(len(valid_tags)):
        if modified_values[i] < scaler.data_min_[i]:
            modified_values[i] = float(scaler.data_min_[i])
        elif modified_values[i] > scaler.data_max_[i]:
            modified_values[i] = float(scaler.data_max_[i])

modified_array = modified_values.reshape(1, -1)
modified_scaled = scaler.transform(modified_array)
decoded_scaled = autoencoder.predict(modified_scaled, verbose=0)

HI_limite = float(np.mean((modified_scaled - decoded_scaled) ** 2))
print(f"HI_limite = {HI_limite:.6f}")
with open("dados/HI_limit.txt", "w") as f:
    f.write(f"{HI_limite:.6f}\n")


HI_limite = 0.865946


/var/folders/gb/ky0h6snd0sd6dqm0djnndhph0000gn/T/ipykernel_67699/2920964095.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  modified_values[i] = apply_rule_to_mean(mean_values[i], palavra_guia, d)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
